# Max Rate Utility Example

This example shows the use of the utility function **get_max_sample_rates** to determine the maximum same rates based on the number of converters (JESD param M) and limitations based on the FPGA itself.

For reference:
- L is the number of lanes used per link and 
- Sample clock is in samples per second
- Bit clock is equilavent to lane rate (bit clock is the technical name for lane rate)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/analogdevicesinc/pyadi-jif/blob/main/examples/ad9081_rx_max_rate_search.ipynb)

In [ ]:
!pip install -r requirements_ex_util_max_rate.txt
!pip install -U -i https://test.pypi.org/simple/ pyadi-jif

In [ ]:
# This example determines the maximum sample rate based on
# FPGA platform and JESD204 class
import adijif as jif
import numpy as np
import pandas as pd
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [ ]:
# Set up Converter and FPGA models
conv = jif.ad9081_rx()

fpga = jif.xilinx()
fpga.setup_by_dev_kit_name("zc706")
fpga.sys_clk_select = "GTH34_SYSCLK_QPLL0"  # Use faster QPLL

results = jif.utils.get_max_sample_rates(conv, fpga)

# Print table
df = pd.DataFrame.from_dict(results)
df

In [ ]:
# Look at results without FPGA requirements
results = jif.utils.get_max_sample_rates(conv)

# Print table
df = pd.DataFrame.from_dict(results)
df